<a href="https://colab.research.google.com/github/tuomaseerola/emr/blob/main/Chapter08_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 8 Score Analysis Code

A Jupyter notebook with code examples in Python for _Chapter 8 (Score Analysis)_ of _Routlegde_ book titled **[How to Conduct Empirical Music Research](https://github.com/tuomaseerola/emr)** by [Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), [Music and Science Lab]() at [Durham University](https://www.durham.ac.uk) and [Music and Science Lab](https://musicscience.net) scheduled to be published in 2023.

To run the code in your browser, open the file in **Colab** (click the icon "Open in Colab"). Alternatively, you can download the notebook and run it locally.

> File `Chapter08_score.ipynb` | 
> Version `4/9/2022` |
> [Back to Index](https://github.com/tuomaseerola/emr)

---

### Install Music21 and other elements needed to run the environment
Press play and wait for all commands to be executed - this initial command might take some time as it needs to build the musi21 environment.

In [ ]:
!pip install --upgrade music21
!add-apt-repository ppa:mscore-ubuntu/mscore-stable -y
!apt-get update
!apt-get install musescore
!apt-get install xvfb
!sh -e /etc/init.d/x11-common start
import os
os.putenv('DISPLAY', ':99.0')
!start-stop-daemon --start --pidfile /var/run/xvfb.pid --make-pidfile --background --exec /usr/bin/Xvfb -- :99 -screen 0 1024x768x24 -ac +extension GLX +render -noreset
from music21 import *
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/mscore'
us['directoryScratch'] = '/tmp'

## Ch6_Ex01.py
Harmonic and metrical analysis of an example excerpt (`bach/bwv30.6`) using `music21`.

In [ ]:
# Ch6_Ex01.py
# Part of EMR 2023, Tuomas Eerola
# Available at: https://github.com/tuomaseerola/emr/

from music21 import *                     # activate library

### 1 Select one example from Bach chorales
bwv30_6 = corpus.parse('bach/bwv30.6.xml')# Take an example
bwv30_6.measures(1, 3).show()             # Display 3 bars
### 2 Harmonic analysis
bChords = bwv30_6.chordify()              # Slice the chords
for c in bChords.recurse().getElementsByClass('Chord'):
    c.closedPosition(forceOctave=4, inPlace=True)
# Run analysis and add Roman numerals as lyrics
for c in bChords.recurse().getElementsByClass('Chord'):
    rn = roman.romanNumeralFromChord(c, key.Key('A'))
    c.addLyric(str(rn.figure))
bChords.measures(0,3).show()             # Display the result
### 3 Metrical analysis
bass = bwv30_6.getElementById('Bass')    # Get the bass part
excerpt = bass.measures(1,3)             # Bar 1 through 3
analysis.metrical.labelBeatDepth(excerpt)# Metrical analysis
excerpt.show()                           # Display the results

## Key-analysis
Key-finding algorithm applied to an example excerpt (`bach/bwv30.6`).

In [ ]:
!pip show music21
import sys
print(sys.version)

In [ ]:
import pandas as pd
bwv30_6 = corpus.parse('bach/bwv30.6.xml')# 30.6
print(bwv30_6.analyze('key.krumhanslkessler'))
bwv30_6_3meas = bwv30_6.measures(1,4) # First 3 measures

KK = analysis.discrete.KrumhanslKessler() # Key profile
wa = analysis.windowed.WindowedAnalysis(bwv30_6_3meas, KK)
a,b = wa.analyze(2, windowType='overlap')

keyclar=[]; mode=[]; key=[]
for x in range(len(a)):
    key.append(a[x][0])
    mode.append(a[x][1])
    keyclar.append(a[x][2])
data=pd.DataFrame({'key':key,'mode':mode,'r':keyclar})
print(data)

## Tension
Analysis of tonal tension using the model by Herremans and Chew (2016), implemented in `partitura` library for Python.

In [ ]:
! pip install partitura
import partitura
import numpy as np


In [ ]:
!wget https://raw.githubusercontent.com/tuomaseerola/music21/master/bwv306.musicxml

In [ ]:
part = partitura.load_musicxml('bwv306.musicxml')
tonal_tension = partitura.musicanalysis.estimate_tonaltension(part, ss='onset')
x = getattr(tonal_tension['onset_beat'][0:50], "tolist", lambda: value)()
y=tonal_tension['cloud_momentum'][0:50]

d = {'beat': x,'tension': y}
df = pd.DataFrame(data=d)
print(df)
